In [1]:
import os
import json
import re
from openai import OpenAI

class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = page_splits[i]
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"第{page}页", para))

    # 从所有段落中均匀采样最多 max_segments 段
    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = "请根据以下文档内容，生成一个问题和答案对，并标明文档页码和原始文本段。\n\n"
    prompt += "【示例1】\n" + few_shot_examples[0] + "\n\n"
    prompt += "【示例2】\n" + few_shot_examples[1] + "\n\n"
    prompt += f"【文档内容】\n{text_segment}\n\n"
    prompt += '''"请生成高质量的问答对（问题+答案），问题应基于下列内容，不抄袭或者基于标题或目录，必须为用户可能提出的真实业务/操作/流程/条款相关问题。
一、问题部分：为同一个主题创建尽可能多的不同表述的问题，确保问题的多样性。
    每个问题应考虑用户可能的多种问法，如：
        - 什么是...？
        - 是否可以说...？
        - 请解释一下...的含义
        - 如果...会怎样？
        - 能否举个例子说明...？
        - 为什么...会发生？
        - ...的好处是什么？

二、答案部分：提供全面、信息丰富的答案，确保逻辑清晰、准确无误。
请输出严格的JSON格式，字段包括：question, answer, 文档来源, 页码, 文本片段。"'''
    return prompt


def main():
    # ===== 用户配置 =====
    input_folder = "D:/desktop/code/data"
    output_file = "output.json"
    qa_per_doc = 3
    api_key = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"
    base_url = "https://open.bigmodel.cn/api/paas/v4"
    model_name = "glm-4-0520"
    # ===================

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第6页",
"文本片段": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第7页",
"文本片段": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            try:
                response = chat_model.chat(prompt)

                # 提取 JSON 块
                match = re.search(r'\{[\s\S]*?\}', response)
                if not match:
                    raise ValueError("找不到有效的JSON对象")
                json_str = match.group(0)

                parsed = json.loads(json_str)
                parsed["文档来源"] = filename
                parsed["页码"] = page
                parsed["文本片段"] = segment
                results.append(parsed)

            except Exception as e:
                print(f"错误 - 文件 {filename} 第{idx+1}段（页码：{page}）：{e}")
                continue

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(results)} 个 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第2段（页码：第16页）：Expecting ',' delimiter: line 8 column 1 (char 307)
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：第33页）：Expecting ',' delimiter: line 8 column 1 (char 278)
错误 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第2段（页码：第8页）：Expecting ',' delimiter: line 8 column 1 (char 203)
错误 - 文件 宏摯傳承保單條款_2024_04.md 第2段（页码：第9页）：Invalid control character at: line 6 column 109 (char 279)
错误 - 文件 宏摯傳承保障計劃_产品彩页.md 第3段（页码：第17页）：Expecting ',' delimiter: line 8 column 1 (char 237)
错误 - 文件 活耀人生危疾保-产品手册.md 第1段（页码：第0页）：Expecting ',' delimiter: line 8 column 3 (char 244)
错误 - 文件 活耀人生危疾保-产品手册.md 第3段（页码：第19页）：Unterminated string starting at: line 6 column 11 (char 211)
错误 - 文件 电子行政运作手册_保单行政_2023_10.md 第2段（页码：第32页）：Expecting ',' delimiter: line 8 column 2 (char 176)
错误 - 文件 电子行政运作手册_保单行政_2023_10.md 第3段（页码：第62页）：Expecting ',' delimiter: line 8 column 1 (char 202)
错误 - 文件 电子行政运作手册_理赔_2024_01.md 第1段（页码：第0页）：Invalid \escape: line 6 column 60 (char 249)
错误 - 文件 电子行政运作手册_理赔_2024_01.md 第2段（页码：第28页）：Expe

In [8]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"第{page}页", para))

    # 从所有段落中均匀采样最多 max_segments 段
    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = "请根据以下文档内容，生成一个问题和答案对，并标明文档页码和原始文本段。输出必须是严格的 JSON 格式，字段包括：question, answer, 文档来源, 页码, 文本片段。\n\n"
    prompt += "【示例1】\n" + few_shot_examples[0] + "\n\n"
    prompt += "【示例2】\n" + few_shot_examples[1] + "\n\n"
    prompt += f"【文档内容】\n{text_segment}\n\n"
    prompt += '''请生成一个高质量的问答对（问题+答案），问题应基于下列内容，不抄袭或者基于标题或目录，
    作为保险条款分析师，生成的问题必须为用户可能提出的真实业务/操作/流程/条款相关问题。
一、问题部分：为同一个主题创建尽可能多的不同表述的问题，确保问题的多样性。
    每个问题应考虑用户可能的多种问法，如：
        - 什么是...？
        - 是否可以说...？
        - 请解释一下...的含义
        - 如果...会怎样？
        - 能否举个例子说明...？
        - 为什么...会发生？
        - ...的好处是什么？

二、答案部分：提供全面、信息丰富的答案，答案不应完全基于文本内容，确保逻辑清晰、准确无误。

请严格按照以下 JSON 格式输出：
{
    "question": "问题内容",
    "answer": "答案内容",
    "文档来源": "文档文件名",
    "页码": "页码",
    "文本片段": "原始文本段"
}'''
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    # ===== 用户配置 =====
    input_folder = "D:/desktop/code/data"
    output_file = "QA.json"
    qa_per_doc = 20
    api_key = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"
    base_url = "https://open.bigmodel.cn/api/paas/v4"
    model_name = "glm-4-0520"
    # ===================

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第6页",
"文本片段": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第7页",
"文本片段": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["文档来源"] = filename
                        parsed["页码"] = page
                        parsed["文本片段"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 6 column 63 (char 257)
重试 1/2 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：第5页）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 6 column 63 (char 267)
重试 2/2 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：第5页）：无法解析 JSON
错误 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：第5页）：达到最大重试次数，无法获取有效数据
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第1段（页码：第3页）：无法解析 JSON
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第6段（页码：第9页）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 11 (char 10)
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第15段（页码：第18页）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 11 (char 10)
重试 2/2 - 文件 活耀人生危疾保-产品手册.md 第15段（页码：第18页）：无法解析 JSON
错误 - 文件 活耀人生危疾保-产品手册.md 第15段（页码：第18页）：达到最大重试次数，无法获取有效数据
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第19段（页码：第23页）：无法解析 JSON

成功保存 138 个高质量 QA 对到 QA.json


In [ ]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"第{page}页", para))

    # 从所有段落中均匀采样最多 max_segments 段
    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，擅长从保险文档中提炼用户关心的问题，并提供准确专业的答案。\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "生成内容基于【文档内容】部分，不得引用文档中未提及的信息。\n\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "文档来源": "文档文件名",\n'
        '    "页码": "页码",\n'
        '    "文本片段": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    # ===== 用户配置 =====
    input_folder = "D:/desktop/code/data"
    output_file = "QA2.json"
    qa_per_doc = 20
    api_key = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"
    base_url = "https://open.bigmodel.cn/api/paas/v4"
    model_name = "glm-4-0520"
    # ===================

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第6页",
"文本片段": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第7页",
"文本片段": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["文档来源"] = filename
                        parsed["页码"] = page
                        parsed["文本片段"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 6 column 63 (char 251)
重试 1/2 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：第5页）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 6 column 63 (char 258)
重试 2/2 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：第5页）：无法解析 JSON
错误 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：第5页）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Extra data: line 1 column 11 (char 10)
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第15段（页码：第18页）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 11 (char 10)
重试 2/2 - 文件 活耀人生危疾保-产品手册.md 第15段（页码：第18页）：无法解析 JSON
错误 - 文件 活耀人生危疾保-产品手册.md 第15段（页码：第18页）：达到最大重试次数，无法获取有效数据

成功保存 138 个高质量 QA 对到 QA2.json


In [ ]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0，1，2页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"第{page}页", para))

    # 从所有段落中均匀采样最多 max_segments 段
    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，问题不应抄袭目录，语言表达应贴近用户常见问法，不要局限于一种问法\n"
        "请使用以下几种问题风格中的任意一种生成问题：\n"
            "- 用户想了解含义或定义\n"
            "- 用户关心具体适用场景\n"
            "- 用户希望了解流程或限制\n"
            "- 用户想判断某种情况是否能获得保障\n"
            "- 用户试图对比两种保障的差异\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力，完整且准确。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "文档来源": "文档文件名",\n'
        '    "页码": "页码",\n'
        '    "文本片段": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    # ===== 用户配置 =====
    input_root_folder = "D:/desktop/code/data"
    output_file = "QA_doubao2.json"
    qa_per_doc = 20
    api_key = "4606b810-d195-4fbc-ba0e-b4249b43fb45"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "doubao-1.5-pro-32k-250115"
    # ===================

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
                "question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
                "answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
                "文档来源": "电子行政运作手册_保单行政_2023_10.md",
                "页码": "第6页",
                "文本片段": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
                "question": "什么情况下保险顾问的营业积分会被扣回？",
                "answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
                "文档来源": "电子行政运作手册_保单行政_2023_10.md",
                "页码": "第7页",
                "文本片段": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for doc_folder in os.listdir(input_root_folder):
        doc_path = os.path.join(input_root_folder, doc_folder)
        if not os.path.isdir(doc_path):
            continue

        # 找到 .md 文件（只取第一个）
        md_file = None
        for file in os.listdir(doc_path):
            if file.endswith(".md"):
                md_file = os.path.join(doc_path, file)
                break

        if not md_file:
            print(f"未找到 Markdown 文件：{doc_folder}")
            continue

        with open(md_file, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["文档来源"] = doc_folder
                        parsed["页码"] = page
                        parsed["文本片段"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {doc_folder} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {doc_folder} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {doc_folder} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 6 column 63 (char 272)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07 第5段（页码：第11页）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 6 column 63 (char 244)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07 第5段（页码：第11页）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07 第5段（页码：第11页）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 6 column 68 (char 233)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07 第15段（页码：第33页）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 6 column 68 (char 225)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07 第15段（页码：第33页）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07 第15段（页码：第33页）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 6 column 187 (char 496)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07 第16段（页码：第35页）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 6 column 187 (char 491)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07 第16段（页码：第35页）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07 第16段（页码：第35页）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 6 column 28 (char 214)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07 第19段（页码：第40页）：无法解析 JSON
JSON 解析错误: Invalid \escape: